In [2]:
import numpy as np
import pandas as pd

## apply

Code that uses `.apply()` looks clean, but it is rather slow when used row-wise (`axis=1`). To quantify this, you can run the example below.

In [31]:
size = 100_000
df = pd.DataFrame({
    'A': np.random.uniform(0.0, 1.0, size=size),
    'B': np.random.uniform(0.0, 1.0, size=size),
})

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   A       100000 non-null  float64
 1   B       100000 non-null  float64
dtypes: float64(2)
memory usage: 1.5 MB


Note that this dataframe is fairly small.

### Evaluating a condition

In [38]:
%timeit df.apply(lambda x: 0 if x.A + x.B < 1.0 else 1, axis=1)

551 ms ± 8.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [39]:
%timeit np.select([df.A + df.B < 1.0, df.A + df.B >= 1.0], [0, 1])

1.17 ms ± 5.24 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [40]:
%timeit np.where(df.A + df.B < 1.0, 0, 1)

510 μs ± 4.17 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Clearly, `.apply()` is very slow comparted to `np.select()` and `np.where()`.  Note that `np.where()` is faster than `np.select()` by a factor of 2.

In [50]:
assert np.array_equal(
    df.apply(lambda x: 0 if x.A + x.B < 1.0 else 1, axis=1).to_numpy(),
    np.where(df.A + df.B < 1.0, 0, 1),
)

In [51]:
assert np.array_equal(
    df.apply(lambda x: 0 if x.A + x.B < 1.0 else 1, axis=1).to_numpy(),
    np.select([df.A + df.B < 1.0, df.A + df.B >= 1.0], [0, 1]),
)

All three approaches produce the same results.

### Adding a column

In [41]:
%timeit df['C'] = df.apply(lambda x: x.A + x.B, axis=1)

563 ms ± 8.58 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [42]:
%timeit df['C'] = df.A + df.B

176 μs ± 2.21 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Clearly, `.apply()` is very slow comparted to a straightforward column definition.  The difference is a factor of 1,000.

In [48]:
assert df.apply(lambda x: x.A + x.B, axis=1).equals(df.A + df.B)

Both approaches yield the same result.

### Aggregating columns

Although less dramatically so, applying `.apply()` along axis 0 is also slower than its numpy counterpart.

In [52]:
%timeit df.apply(np.sum, axis=0)

303 μs ± 4.28 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [54]:
%timeit np.sum(df.to_numpy(), axis=0)

179 μs ± 10.2 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [55]:
assert np.array_equal(df.apply(np.sum, axis=0), np.sum(df.to_numpy(), axis=0))

Again, both produce the same result.